# Lesson 2 - motor and scan

In this lesson, we'll work with a motor (for precise positioning) in [Bluesky](http://nsls-ii.github.io/bluesky/) and [related tools](http://nsls-ii.github.io/).

First, we'll connect with an EPICS motor (using [ophyd](http://nsls-ii.github.io/ophyd/)), and then use the Bluesky software to scan the motor (with the scaler from [lesson 1](lesson1.ipynb).

Load `ophyd` device support for the `EpicsMotor` and connect with one EPICS motor channel.  We have a synApps (v6.1) XXX-style IOC with the prefix `sky:`. It has a scaler, 16 soft channel motors, and some other support we'll ignore in this lesson.

In [1]:
from ophyd import EpicsMotor
m1 = EpicsMotor("sky:m1", name="m1")
m1.wait_for_connection()

Show the current value fo the motor (that's the `.RBV` field, in case you were interested).

In [2]:
m1.position

5.0

Connect the scaler (as was done in lesson 1).  Define some of the chaannel names and clear out others.

In [3]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH("sky:scaler1", name="scaler")
scaler.wait_for_connection()

# Since there are no detectors actually connected to this scaler,
# we can change names at our choice.  A real scaler will have
# detectors connected to specific channels and we should not modify
# these names without regard to how signals are physically connected.
scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")
scaler.channels.chan04.chname.put("")
scaler.channels.chan05.chname.put("")
scaler.channels.chan06.chname.put("")
scaler.channels.chan07.chname.put("")
scaler.channels.chan08.chname.put("")
scaler.channels.chan09.chname.put("")

Use only the channels with EPICS names, those are the *interesting* channels.

In [4]:
from apstools.devices import use_EPICS_scaler_channels

scaler.match_names()
use_EPICS_scaler_channels(scaler)
scaler.read()

OrderedDict([('clock', {'value': 5000000.0, 'timestamp': 1587534493.32884}),
             ('I0', {'value': 2.0, 'timestamp': 1587534493.32884}),
             ('scint', {'value': 1.0, 'timestamp': 1587534493.32884}),
             ('scaler_time', {'value': 0.5, 'timestamp': 1587534493.32884})])

Create a RunEngine but do not connect it with a data collection strategy.  That will come in the next lessons.

In [5]:
from bluesky import RunEngine
import bluesky.plans as bp
RE = RunEngine({})

Run a step scan using the motor and the scaler.

In [6]:
RE(bp.scan([scaler], m1, -1, 1, 5))

('444eac01-8329-4443-9a94-b7bba1c0f91b',)

Ah, yes.  Nothing to see here since we did not setup anything to receive the *documents* from the RunEngine.  Here's the basic callback from lesson 1.

In [7]:
def myCallback(key, doc):
    print(key, len(doc))
    for k, v in doc.items():
        print("\t", k, v)

Repeat the same scan but handle the document stream with `myCallback()`.

In [8]:
RE(bp.scan([scaler], m1, -1, 1, 5), myCallback)

start 15
	 uid 8fd6dfc2-202e-4426-b1f1-905664bcb1cd
	 time 1587534618.9915764
	 versions {'ophyd': '1.4.0rc4', 'bluesky': '1.6.0rc4'}
	 scan_id 2
	 plan_type generator
	 plan_name scan
	 detectors ['scaler']
	 motors ('m1',)
	 num_points 5
	 num_intervals 4
	 plan_args {'detectors': ["ScalerCH(prefix='sky:scaler1', name='scaler', read_attrs=['channels', 'channels.chan01', 'channels.chan01.s', 'channels.chan02', 'channels.chan02.s', 'channels.chan03', 'channels.chan03.s', 'time'], configuration_attrs=['channels', 'channels.chan01', 'channels.chan01.chname', 'channels.chan02', 'channels.chan02.gate', 'channels.chan03', 'channels.chan03.chname', 'channels.chan03.preset', 'count_mode', 'delay', 'auto_count_delay', 'freq', 'preset_time', 'auto_count_time', 'egu'])"], 'num': 5, 'args': ["EpicsMotor(prefix='sky:m1', name='m1', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_

('8fd6dfc2-202e-4426-b1f1-905664bcb1cd',)

# Summary

We'll show this code as a python program:

```
#!/APSshare/anaconda3/BlueSky/bin/python

"lesson 2: motor"

from ophyd import EpicsMotor
from ophyd.scaler import ScalerCH
from bluesky import RunEngine
import bluesky.plans as bp
from apstools.devices import use_EPICS_scaler_channels


def myCallback(key, doc):
    print(key, len(doc))
    for k, v in doc.items():
        print("\t", k, v)
    print("~~~~~~~~~~~~~~~~~")


m1 = EpicsMotor("sky:m1", name="m1")
m1.wait_for_connection()
print(m1.position)

scaler = ScalerCH("sky:scaler1", name="scaler")
scaler.wait_for_connection()


# Since there are no detectors actually connected to this scaler,
# we can change names at our choice.  A real scaler will have
# detectors connected to specific channels and we should not modify
# these names without regard to how signals are physically connected.
scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")
scaler.channels.chan04.chname.put("")
scaler.channels.chan05.chname.put("")
scaler.channels.chan06.chname.put("")
scaler.channels.chan07.chname.put("")
scaler.channels.chan08.chname.put("")
scaler.channels.chan09.chname.put("")


scaler.match_names()
use_EPICS_scaler_channels(scaler)
print(scaler.read())

RE = RunEngine({})

RE(bp.scan([scaler], m1, -1, 1, 5))

RE(bp.scan([scaler], m1, -1, 1, 5), myCallback)
```